# _**GoogLeNet_voc2012**


1. inception 모듈
2. auxiliary classifier 
3. global average pooling

![alt text](image-1.png)

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import os


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")

Using device: cuda
CUDA available: True


In [15]:
import torch
import torch.nn as nn
from torch import Tensor
from typing import Optional

In [16]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs) -> None:
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        return x

In [17]:

class Inception(nn.Module):
    def __init__(self, in_channels, n1x1, n3x3_reduce, n3x3, n5x5_reduce, n5x5, pool_proj) -> None:
        super(Inception, self).__init__()
        self.branch1 = ConvBlock(in_channels, n1x1, kernel_size=1, stride=1, padding=0)

        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, n3x3_reduce, kernel_size=1, stride=1, padding=0),
            ConvBlock(n3x3_reduce, n3x3, kernel_size=3, stride=1, padding=1))
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, n5x5_reduce, kernel_size=1, stride=1, padding=0),
            ConvBlock(n5x5_reduce, n5x5, kernel_size=5, stride=1, padding=2))

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels, pool_proj, kernel_size=1, stride=1, padding=0))
        
    def forward(self, x: Tensor) -> Tensor:
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        return torch.cat([x1, x2, x3, x4], dim=1)

In [29]:
# 예시 입력 텐서 (배치 크기 2, 채널 3, 높이 32, 너비 32)
a = torch.randn(2, 3, 32, 32)  # 크기: (2, 3, 32, 32)
b = torch.randn(2, 4, 32, 32)  # 크기: (2, 3, 32, 32)
torch.cat([a,b], dim=1)

tensor([[[[-0.8635,  0.4031, -0.0955,  ...,  0.3380, -0.8318,  0.0225],
          [ 0.2375, -1.1175,  0.5509,  ..., -0.0074,  0.1201, -0.7904],
          [-0.4690,  0.0379, -1.0364,  ...,  0.1150, -0.4090,  0.4462],
          ...,
          [-0.4487,  0.1758, -0.8509,  ..., -0.8205, -0.9048,  0.2880],
          [ 1.1155, -0.2393,  1.1096,  ...,  1.4236,  0.2564, -0.4669],
          [ 1.9245, -1.4698,  1.0380,  ...,  0.6034, -0.1941,  0.0756]],

         [[ 0.5930,  0.4838, -0.9952,  ..., -0.0972,  0.0205,  0.8000],
          [-1.6151, -0.1978,  1.1477,  ..., -0.1319, -0.2770,  0.0205],
          [ 0.7238, -0.0265, -0.8581,  ..., -0.4399, -1.3675, -0.0685],
          ...,
          [-1.6555,  1.3695,  1.0087,  ...,  0.5884, -2.3705, -0.1487],
          [-0.1795, -0.8887, -2.0101,  ..., -0.3503,  1.3519, -0.4294],
          [ 0.4794, -1.1205, -0.8351,  ..., -0.4221,  0.5501, -0.1000]],

         [[-0.1722, -1.6042,  0.3616,  ..., -0.3282, -0.7696, -0.0505],
          [ 2.2464, -1.0506,  

In [18]:
input_tensor = torch.randn(2, 3, 32, 32)

# Inception 모델 초기화
model = Inception(in_channels=3, n1x1=64, n3x3_reduce=64, n3x3=128, n5x5_reduce=32, n5x5=32, pool_proj=32)

# 출력
output_tensor = model(input_tensor)

# 결과 출력
print("입력 텐서 크기: ", input_tensor.shape)
print("출력 텐서 크기: ", output_tensor.shape)


입력 텐서 크기:  torch.Size([2, 3, 32, 32])
출력 텐서 크기:  torch.Size([2, 256, 32, 32])


In [19]:
conv = nn.Conv2d(3, 1, kernel_size=2, stride=1, padding=0)
input_tensor = torch.randn(1, 3, 6, 6)
output_tensor = conv(input_tensor)

print("Conv2d Output Shape: ", output_tensor.shape)

pool = nn.AvgPool2d(kernel_size=3, stride=1) # 입력차원과 아웃차원을 명시안함
output_tensor = pool(input_tensor)
print("AvgPool2d Output Shape: ", output_tensor.shape)

output_tensor = conv(output_tensor)
print("Conv2d Output Shape: ", output_tensor.shape)

Conv2d Output Shape:  torch.Size([1, 1, 5, 5])
AvgPool2d Output Shape:  torch.Size([1, 3, 4, 4])
Conv2d Output Shape:  torch.Size([1, 1, 3, 3])


In [20]:
class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes) -> None:
        super(InceptionAux, self).__init__()
        self.avgpool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = ConvBlock(in_channels, 128, kernel_size=1, stride=1, padding=0)
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(p=0.7)
        self.relu = nn.ReLU()

    def forward(self, x: Tensor) -> Tensor:
        x = self.avgpool(x)
        x = self.conv(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [21]:
import torch
import torch.nn as nn
from torch import Tensor

class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes) -> None:
        super(InceptionAux, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # 출력 크기를 (1,1)로 고정
        self.conv = ConvBlock(in_channels, 128, kernel_size=1, stride=1, padding=0)
        self.fc1 = None  # 이후 forward에서 동적으로 생성
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(p=0.7)
        self.relu = nn.ReLU()

    def forward(self, x: Tensor) -> Tensor:
        x = self.avgpool(x)  # (B, C, 1, 1)로 변환
        x = self.conv(x)
        x = x.view(x.shape[0], -1)  # (B, C*1*1) -> (B, C)

        if self.fc1 is None:  # 첫 실행 시 동적으로 생성
            self.fc1 = nn.Linear(x.shape[1], 1024).to(x.device)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [22]:
class GoogLeNet(nn.Module):
    def __init__(self, aux_logits=True, num_classes=1000) -> None:
        # 네트워크 초기화 함수
        super(GoogLeNet, self).__init__()

        # aux_logits이 True이면 보조 분류기 사용, False이면 사용 안함
        assert aux_logits == True or aux_logits == False # optrion을 두가지로 고정
        self.aux_logits = aux_logits

        # 첫 번째 컨볼루션 블록 (64 채널, 커널 크기 7x7, 스트라이드 2)
        self.conv1 = ConvBlock(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3)
        # 첫 번째 맥스풀링 (커널 크기 3x3, 스트라이드 2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True)
        # 두 번째 컨볼루션 블록 (64 채널, 커널 크기 1x1)
        self.conv2 = ConvBlock(in_channels=64, out_channels=64, kernel_size=1, stride=1, padding=0)
        # 세 번째 컨볼루션 블록 (192 채널, 커널 크기 3x3)
        self.conv3 = ConvBlock(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1)
        # 두 번째 맥스풀링
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        # Inception 블록 (세부적인 채널, 커널 크기 등은 Inception 모듈에서 설정)
        self.a3 = Inception(192, 64, 96, 128, 16, 32, 32)
        self.b3 = Inception(256, 128, 128, 192, 32, 96, 64)
        # 세 번째 맥스풀링
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0, ceil_mode=True)
        
        # 4번부터 5번까지 여러 Inception 블록 (GoogLeNet의 핵심)
        self.a4 = Inception(480, 192, 96, 208, 16, 
                            48, 64)
        self.b4 = Inception(512, 160, 112, 224, 24, 
                            64, 64)
        self.c4 = Inception(512, 128, 128, 256, 24, 
                            64, 64)
        self.d4 = Inception(512, 112, 144, 288, 32, 
                            64, 64)
        self.e4 = Inception(528, 256, 160, 320, 32, 
                            128, 128)
        
        # 네 번째 맥스풀링
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # 마지막 Inception 블록
        self.a5 = Inception(832, 256, 160, 320, 32, 128, 128)
        self.b5 = Inception(832, 384, 192, 384, 48, 128, 128)
        
        # Adaptive 평균 풀링 (출력 크기: 1x1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # 드롭아웃 (확률 0.4)
        self.dropout = nn.Dropout(p=0.4)
        # 최종 완전 연결층 (출력 크기: num_classes)
        self.linear = nn.Linear(1024, num_classes)

        # 보조 분류기 (auxiliary classifiers) 설정
        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)
        else:
            self.aux1 = None
            self.aux2 = None

    def transform_input(self, x: Tensor) -> Tensor:
        # 입력 이미지에 대한 정규화 (RGB 채널에 대해 각각 정규화)
        x_R = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
        x_G = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
        x_B = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
        x = torch.cat([x_R, x_G, x_B], 1)
        return x
        
    def forward(self, x: Tensor) -> Tensor:
        # 입력 데이터에 대한 전처리
        x = self.transform_input(x)

        # 컨볼루션 연산 및 풀링 연산을 순차적으로 적용
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool2(x)
        x = self.a3(x)
        x = self.b3(x)
        x = self.maxpool3(x)
        x = self.a4(x)
        
        # 보조 분류기 1 (훈련 중에만 적용)
        aux1: Optional[Tensor] = None # aux1은 Tensor 타입이거나 None일 수 있다.
        if self.aux_logits and self.training:
            aux1 = self.aux1(x) # aux_logits가 True이고 훈련 중이면 aux1을 self.aux1(x)로 설정

        x = self.b4(x)
        x = self.c4(x)
        x = self.d4(x)

        # 보조 분류기 2 (훈련 중에만 적용)
        aux2: Optional[Tensor] = None
        if self.aux_logits and self.training: # self.traing 모델이 훈련중일때만 작동
            aux2 = self.aux2(x)

        x = self.e4(x)
        x = self.maxpool4(x)
        x = self.a5(x)
        x = self.b5(x)
        # Adaptive Avg Pooling 적용 (출력 크기: 1x1)
        x = self.avgpool(x)
        # 텐서 차원을 일렬로 펼침
        x = x.view(x.shape[0], -1)  # x = x.reshape(x.shape[0], -1)
        # 최종 완전 연결층
        x = self.linear(x)
        # 드롭아웃 적용
        x = self.dropout(x)

        # 훈련 중일 때 보조 분류기 결과 반환, 그렇지 않으면 최종 결과 반환
        if self.aux_logits and self.training:
            return aux1, aux2
        else:
            return x


In [23]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 이미지 변환 설정_224x224로 resize, 정규화
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# CIFAR-10 데이터셋 다운로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# DataLoader 설정 / 배치사이즈 32
train_loader = DataLoader(trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(testset, batch_size=32, shuffle=False)

# 데이터셋의 첫 번째 이미지와 라벨 확인
data_iter = iter(train_loader)  # DataLoader 객체를 반복 가능한 객체로 변환
images, labels = next(data_iter)  # 첫 번째 배치를 가져오기

# 라벨 출력
print("라벨:", labels)


Files already downloaded and verified
Files already downloaded and verified
라벨: tensor([2, 2, 1, 6, 6, 3, 7, 7, 3, 8, 4, 9, 3, 6, 6, 5, 1, 7, 8, 7, 3, 2, 9, 5,
        2, 2, 4, 1, 7, 4, 2, 6])


In [24]:

# 모델 생성
def train(model, train_loader, test_loader, num_epochs=10, learning_rate=0.001):
    from tqdm import tqdm
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # CUDA 사용 가능하면 GPU 사용, 아니면 CPU 사용
    model = model.to(device)  # 모델을 지정된 장치(GPU/CPU)로 이동
    
    criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류 손실 함수
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # 모델 학습 및 검증
    for epoch in tqdm(range(num_epochs)):
        model.train()  # 모델을 학습 모드로 전환
        running_loss = 0.0
        
        # 미니 배치 단위로 데이터를 불러옴
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # 입력 데이터를 모델에 통과시킴
            if model.aux_logits:  # aux_logits가 True일 때
                outputs = model(inputs)  # 보조 출력을 포함한 모델 통과
                if isinstance(outputs, tuple):  # 모델이 튜플을 반환하면 보조 출력도 포함
                    outputs, aux1 = outputs  # 보조 출력이 하나만 반환되는 경우
                    aux1 = aux1.view(-1, 10)
                    loss1 = criterion(aux1, targets)
                    loss = criterion(outputs, targets) + 0.3 * loss1
                    # 보조 분류기는 중간 레이어에서 학습을 촉진하는 데 도움이 되며, 이 손실을 주 손실에 추가하여 모델이 더 잘 학습할 수 있도록 만듭니다.
                    # 0.3은 하이퍼파라미터로, 보조 분류기의 손실이 주 손실에 미치는 영향을 조절합니다.
                else:  # 보조 출력을 하나만 반환할 경우
                    outputs = outputs[0]  # 주 출력만 사용
                    targets = targets.view(-1)  # (batch_size * height * width,)
                    outputs = outputs.view(-1, 10)
                    loss = criterion(outputs, targets)
            else:  # aux_logits가 False일 때
                outputs = model(inputs)
                targets = targets.view(-1)  # (batch_size * height * width,)
                outputs = outputs.view(-1, 10)  # (batch_size * height * width, num_classes)
                loss = criterion(outputs, targets)

            # 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        
        # 검증 데이터로 성능 평가
        evaluate(model, test_loader, device)

# 검증 함수
def evaluate(model, test_loader, device):
    model.eval()  # 모델을 평가 모드로 전환
    correct = 0
    total = 0
    # 그라디언트 계산 비활성화
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # 입력 데이터를 지정된 장치로 이동
            outputs = model(inputs)
            outputs = outputs.view(-1, 10)  # (batch_size * height * width, num_classes)
            targets = targets.view(-1)  # (batch_size * height * width,)
            
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")






In [28]:
# 모델 생성 및 학습
model = GoogLeNet(aux_logits=False, num_classes=10)  # CIFAR10에는 10개의 클래스를 사용합니다
train(model, train_loader, test_loader, num_epochs=10, learning_rate=0.001)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Loss: 1.9054


 10%|█         | 1/10 [01:24<12:37, 84.20s/it]

Validation Accuracy: 50.74%
Epoch [2/10], Loss: 1.5536


 20%|██        | 2/10 [02:50<11:21, 85.23s/it]

...

Validation Accuracy: 81.88%
Epoch [9/10], Loss: 0.9468


 90%|█████████ | 9/10 [12:40<01:24, 84.38s/it]

Validation Accuracy: 83.22%
Epoch [10/10], Loss: 0.9149


100%|██████████| 10/10 [14:04<00:00, 84.43s/it]

Validation Accuracy: 83.30%
